In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from io import BytesIO
import os

In [7]:
train_dir = "horse-or-human"
os.listdir(train_dir)

['humans', 'horses']

In [11]:
horse_train_dir = os.path.join(train_dir,"horses")
human_train_dir = os.path.join(train_dir,"humans")

In [12]:
train_horse_names = os.listdir(horse_train_dir)
print(f"5 files in horses subdir: {train_horse_names[:5]}")
train_human_names = os.listdir(human_train_dir)
print(f"5 files in humans subdir:{train_human_names[:5]}")

5 files in horses subdir: ['horse43-5.png', 'horse06-5.png', 'horse20-6.png', 'horse04-7.png', 'horse41-7.png']
5 files in humans subdir:['human17-22.png', 'human10-17.png', 'human10-03.png', 'human07-27.png', 'human09-22.png']


In [13]:
print(f"Number of horse images to train: {len(os.listdir(horse_train_dir))}")
print(f"Number of human images to train: {len(os.listdir(human_train_dir))}")

Number of horse images to train: 500
Number of human images to train: 527
